In [69]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import scipy
from scipy import stats
from matplotlib import pyplot as plt
# loading original module
import NeuroProcessing

In [70]:
deaf_files_path = Path("./deaf_datas")
deaf_files = list(deaf_files_path.glob("*"))
intact_files_path = Path("./intact_datas/")
intact_files = list(intact_files_path.glob("*"))

In [46]:
#pdのファイルの結合
deaf_datas = pd.concat([pd.read_csv(deaf_file) for deaf_file in deaf_files])
intact_datas = pd.concat([pd.read_csv(intact_file) for intact_file in intact_files])

### 各刺激に対して通常個体と難聴個体の有意差検定を行う
#### 終わり次第必要なデータをまとめてbarplotを行う

In [71]:
deaf_datas = deaf_datas.dropna()
intact_datas = intact_datas.dropna()

In [48]:
deaf_datas.head()

,date,filename,stim_params,abr_z_score,abr_latency,lfp_z_score_100,lfp_latency_100,lfp_ampliture_100,lfp_z_score_200,lfp_latency_200,...,lfp_ampliture_500,lfp_z_score_600,lfp_latency_600,lfp_ampliture_600,lfp_z_score_700,lfp_latency_700,lfp_ampliture_700,lfp_z_score_800,lfp_latency_800,lfp_ampliture_800
0,20221012,06_us_cont_10ms,150V_d_10.0ms_f_500.0kHz_w_10%,3.799270,3.52,31.524712,32.0,-108.988127,29.757842,32.0,...,-107.921793,43.938221,28.0,-87.103936,47.894683,49.0,-75.497163,49.310815,51.0,-68.041137
1,20221012,06_us_cont_10ms,150V_d_10.0ms_f_500.0kHz_w_25%,0.991067,1.20,9.707665,34.0,-53.076837,22.866414,34.0,...,-51.456555,17.754061,54.0,-48.477232,36.849874,53.0,-43.982124,33.727040,59.0,-38.755543
2,20221012,06_us_cont_10ms,150V_d_10.0ms_f_500.0kHz_w_0%,3.520575,3.36,1.456868,55.0,-3.074728,1.579618,57.0,...,-0.052503,1.672644,59.0,-1.918998,2.541181,12.0,-2.632387,2.049035,59.0,-3.154796
3,20221012,15_us_after_overdose,150V_d_5.0ms_f_500.0kHz_w_0%,4.551804,3.20,1.643152,59.0,-6.621839,1.771019,16.0,...,-5.964583,1.455826,16.0,-6.093422,1.706076,62.0,-5.471790,1.693284,62.0,-5.521663
4,20221012,15_us_after_overdose,150V_d_13.4ms_f_500.0kHz_w_25%_prf_1500Hz_pd_0us,1.528935,3.76,1.616005,91.0,-6.357631,1.388682,48.0,...,-6.007332,1.393090,0.0,-6.378411,1.415158,0.0,-5.053806,1.371154,91.0,-5.315640


In [49]:
intact_datas.head()

,date,filename,stim_params,abr_z_score,abr_latency,lfp_z_score_100,lfp_latency_100,lfp_ampliture_100,lfp_z_score_200,lfp_latency_200,...,lfp_ampliture_500,lfp_z_score_600,lfp_latency_600,lfp_ampliture_600,lfp_z_score_700,lfp_latency_700,lfp_ampliture_700,lfp_z_score_800,lfp_latency_800,lfp_ampliture_800
0,20220804,05_us_burst_and_sham,sham,0.801040,1.84,3.271857,94.0,-12.326964,3.291230,97.0,...,-2.890858,0.474498,68.0,-2.091701,2.838106,94.0,-10.735775,3.101988,97.0,-11.638241
1,20220804,05_us_burst_and_sham,150V_d_6.7ms_f_500.0kHz_w_25%_prf_1500Hz_pd_0us,18.493904,1.24,44.360058,27.0,-209.640918,39.103277,29.0,...,-126.162280,16.483213,32.0,-74.322771,24.672420,29.0,-123.922741,37.789527,27.0,-180.154051
2,20220804,05_us_burst_and_sham,150V_d_6.7ms_f_500.0kHz_w_0%_prf_1500Hz_pd_0us,14.196369,3.40,2.340018,22.0,1.694793,2.936386,22.0,...,8.325145,1.718815,22.0,3.204346,3.471916,53.0,-1.883173,2.336542,22.0,0.824477
3,20220804,07_us_cont_20ms,150V_d_20.0ms_f_500.0kHz_w_10%,2.166277,2.48,28.026349,31.0,-129.806578,28.428800,32.0,...,-85.152949,13.643226,34.0,-59.061607,21.929685,51.0,-92.043153,25.131364,29.0,-110.192800
4,20220804,07_us_cont_20ms,150V_d_20.0ms_f_500.0kHz_w_25%,0.776279,1.28,28.509447,33.0,-122.826127,26.484052,35.0,...,-68.319002,10.545281,35.0,-46.897918,19.218929,32.0,-77.619144,24.320507,32.0,-104.779195


In [50]:
alive_deaf_data = deaf_datas.loc[~(deaf_datas["filename"].str.contains("overdose"))]
alive_intact_data = intact_datas.loc[~(intact_datas["filename"].str.contains("overdose"))]
alive_deaf_data.head()

,date,filename,stim_params,abr_z_score,abr_latency,lfp_z_score_100,lfp_latency_100,lfp_ampliture_100,lfp_z_score_200,lfp_latency_200,...,lfp_ampliture_500,lfp_z_score_600,lfp_latency_600,lfp_ampliture_600,lfp_z_score_700,lfp_latency_700,lfp_ampliture_700,lfp_z_score_800,lfp_latency_800,lfp_ampliture_800
0,20221012,06_us_cont_10ms,150V_d_10.0ms_f_500.0kHz_w_10%,3.799270,3.52,31.524712,32.0,-108.988127,29.757842,32.0,...,-107.921793,43.938221,28.0,-87.103936,47.894683,49.0,-75.497163,49.310815,51.0,-68.041137
1,20221012,06_us_cont_10ms,150V_d_10.0ms_f_500.0kHz_w_25%,0.991067,1.20,9.707665,34.0,-53.076837,22.866414,34.0,...,-51.456555,17.754061,54.0,-48.477232,36.849874,53.0,-43.982124,33.727040,59.0,-38.755543
2,20221012,06_us_cont_10ms,150V_d_10.0ms_f_500.0kHz_w_0%,3.520575,3.36,1.456868,55.0,-3.074728,1.579618,57.0,...,-0.052503,1.672644,59.0,-1.918998,2.541181,12.0,-2.632387,2.049035,59.0,-3.154796
8,20221012,02_ex_click,40dB,1.009564,3.80,3.644626,98.0,-1.551409,5.426701,99.0,...,-3.111724,8.278348,99.0,-3.798667,8.487182,99.0,-2.127918,8.387878,99.0,-2.259726
9,20221012,02_ex_click,60dB,4.211003,3.92,1.291839,99.0,-8.235590,1.708682,98.0,...,-9.348235,2.470178,99.0,-10.035177,4.697765,98.0,-10.522627,5.466739,99.0,-10.109393


In [51]:
deaf_each_stim_data = alive_deaf_data.groupby(by=["stim_params"])
intact_each_stim_data = alive_intact_data.groupby(by = ["stim_params"])
deaf_each_stim_data.describe()

date                \
                                                 count          mean   
stim_params                                                            
150V_d_0.5ms_f_500.0kHz_w_0%                       3.0  2.022112e+07   
150V_d_0.5ms_f_500.0kHz_w_10%                      3.0  2.022112e+07   
150V_d_0.5ms_f_500.0kHz_w_25%                      4.0  2.022109e+07   
150V_d_1.0ms_f_500.0kHz_w_0%                       3.0  2.022112e+07   
150V_d_1.0ms_f_500.0kHz_w_10%                      3.0  2.022112e+07   
150V_d_1.0ms_f_500.0kHz_w_25%                      6.0  2.022113e+07   
150V_d_10.0ms_f_500.0kHz_w_0%                      3.0  2.022112e+07   
150V_d_10.0ms_f_500.0kHz_w_10%                     3.0  2.022112e+07   
150V_d_10.0ms_f_500.0kHz_w_25%                     7.0  2.022112e+07   
150V_d_13.4ms_f_500.0kHz_w_0%_prf_1500Hz_pd_0us    1.0  2.022101e+07   
150V_d_13.4ms_f_500.0kHz_w_25%_prf_1500Hz_pd_0us   2.0  2.022106e+07   
150V_d_20.0ms_f_500.0kHz_w_0%                      4.0  2.022112e+07   
150V_d_20.0ms_f_500.0kHz_w_10%                     4.0  2.022112e+07   
150V_d_20.0ms_f_500.0kHz_w_25%                     3.0  2.022112e+07   
150V_d_5.0ms_f_500.0kHz_w_0%                       2.0  2.022117e+07   
150V_d_5.0ms_f_500.0kHz_w_10%                      3.0  2.022115e+07   
150V_d_5.0ms_f_500.0kHz_w_25%                      2.0  2.022117e+07   
150V_d_6.7ms_f_500.0kHz_w_0%_prf_1500Hz_pd_0us     1.0  2.022101e+07   
150V_d_6.7ms_f_500.0kHz_w_25%_prf_1500Hz_pd_0us    2.0  2.022111e+07   
150V_d_66.7ms_f_500.0kHz_w_25%_prf_1500Hz_pd_0us   1.0  2.022112e+07   
200V_d_13.4ms_f_500.0kHz_w_25%_prf_1500Hz_pd_0us   1.0  2.022112e+07   
200V_d_66.7ms_f_500.0kHz_w_25%_prf_1500Hz_pd_0us   1.0  2.022112e+07   
20dB                                               4.0  2.022112e+07   
300V_d_10.0ms_f_500.0kHz_w_25%_prf_6000Hz_pd_0us   2.0  2.022101e+07   
300V_d_13.4ms_f_500.0kHz_w_25%_prf_1500Hz_pd_0us   1.0  2.022101e+07   
300V_d_5.0ms_f_500.0kHz_w_0%                       2.0  2.022101e+07   
300V_d_5.0ms_f_500.0kHz_w_25%                      3.0  2.022101e+07   
300V_d_6.7ms_f_500.0kHz_w_25%_prf_1500Hz_pd_0us    1.0  2.022101e+07   
30dB                                               4.0  2.022112e+07   
330V_d_6.7ms_f_500.0kHz_w_25%_prf_1500Hz_pd_0us    1.0  2.022122e+07   
40dB                                               4.0  2.022112e+07   
50dB                                               4.0  2.022112e+07   
60dB                                               4.0  2.022112e+07   
70dB                                               3.0  2.022115e+07   
sham                                               5.0  2.022114e+07   

                                                                          \
                                                         std         min   
stim_params                                                                
150V_d_0.5ms_f_500.0kHz_w_0%                      102.026140  20221012.0   
150V_d_0.5ms_f_500.0kHz_w_10%                     102.026140  20221012.0   
150V_d_0.5ms_f_500.0kHz_w_25%                      98.025507  20221012.0   
150V_d_1.0ms_f_500.0kHz_w_0%                      102.026140  20221012.0   
150V_d_1.0ms_f_500.0kHz_w_10%                     102.026140  20221012.0   
150V_d_1.0ms_f_500.0kHz_w_25%                      76.281059  20221012.0   
150V_d_10.0ms_f_500.0kHz_w_0%                     102.026140  20221012.0   
150V_d_10.0ms_f_500.0kHz_w_10%                    102.026140  20221012.0   
150V_d_10.0ms_f_500.0kHz_w_25%                     83.310092  20221012.0   
150V_d_13.4ms_f_500.0kHz_w_0%_prf_1500Hz_pd_0us          NaN  20221012.0   
150V_d_13.4ms_f_500.0kHz_w_25%_prf_1500Hz_pd_0us   74.953319  20221012.0   
150V_d_20.0ms_f_500.0kHz_w_0%                      83.314665  20221012.0   
150V_d_20.0ms_f_500.0kHz_w_10%                     83.314665  20221012.0   
150V_d_20.0ms_f_500.0kHz_w_25%                    102.026140  20221012.0   
150V_d_5.0ms_f_500.0k

In [52]:
#刺激パラメータを配列として取り出して、各パラメータの有意差を出力する
#出力した有意差を元に棒グラフを作成
os.makedirs(f"./result/{deaf_files_path.name}",exist_ok=True)
os.makedirs(f"./result/{intact_files_path.name}",exist_ok=True)

In [53]:
compare_parameters = list(deaf_datas.columns[3:])
compare_parameters

['abr_z_score',
 'abr_latency',
 'lfp_z_score_100',
 'lfp_latency_100',
 'lfp_ampliture_100',
 'lfp_z_score_200',
 'lfp_latency_200',
 'lfp_ampliture_200',
 'lfp_z_score_300',
 'lfp_latency_300',
 'lfp_ampliture_300',
 'lfp_z_score_400',
 'lfp_latency_400',
 'lfp_ampliture_400',
 'lfp_z_score_500',
 'lfp_latency_500',
 'lfp_ampliture_500',
 'lfp_z_score_600',
 'lfp_latency_600',
 'lfp_ampliture_600',
 'lfp_z_score_700',
 'lfp_latency_700',
 'lfp_ampliture_700',
 'lfp_z_score_800',
 'lfp_latency_800',
 'lfp_ampliture_800']

In [57]:
#とりあえずdeafでやる
deaf_stim_parameters = alive_deaf_data["stim_params"].drop_duplicates()
p_value_df = pd.DataFrame(index = [],columns = ["stim_params",*compare_parameters])
for i,stim_parameter in enumerate(deaf_stim_parameters):
    each_stim_deaf = alive_deaf_data.loc[alive_deaf_data["stim_params"] == stim_parameter].dropna(how="all")
    each_stim_intact = alive_intact_data.loc[alive_intact_data["stim_params"] == stim_parameter].dropna(how = "all")
    result_list = [stim_parameter]
    if len(each_stim_intact) < 2:
        continue
    for each_compare_parameter in compare_parameters:
        each_parameter_deaf = each_stim_deaf[each_compare_parameter]
        each_parameter_intact = each_stim_intact[each_compare_parameter]
        result = stats.mannwhitneyu(each_parameter_deaf,each_parameter_intact)
        result_list.append(result.pvalue)
    p_value_df.loc[i]= result_list
p_value_df.to_csv("./result/p_value_at_same_parameter.csv")

In [58]:
# 棒グラフで比較を行う
# 比較したいデータをリストに配置
target_params=["40dB","150V_d_0.5ms_f_500.0kHz_w_0%","150V_d_0.5ms_f_500.0kHz_w_25%",
               "150V_d_20.0ms_f_500.0kHz_w_0%","150V_d_20.0ms_f_500.0kHz_w_25%","sham"]

In [61]:
#各パラメータに対して棒グラフを作成する
target_deaf_data = alive_deaf_data.loc[alive_deaf_data["stim_params"].str.contains("|".join(target_params))]
target_intact_data = alive_intact_data.loc[alive_intact_data["stim_params"].str.contains("|".join(target_params))]
target_deaf_data

,date,filename,stim_params,abr_z_score,abr_latency,lfp_z_score_100,lfp_latency_100,lfp_ampliture_100,lfp_z_score_200,lfp_latency_200,...,lfp_ampliture_500,lfp_z_score_600,lfp_latency_600,lfp_ampliture_600,lfp_z_score_700,lfp_latency_700,lfp_ampliture_700,lfp_z_score_800,lfp_latency_800,lfp_ampliture_800
8,20221012,02_ex_click,40dB,1.009564,3.80,3.644626,98.0,-1.551409,5.426701,99.0,...,-3.111724,8.278348,99.0,-3.798667,8.487182,99.0,-2.127918,8.387878,99.0,-2.259726
13,20221012,04_us_cont_20ms,150V_d_20.0ms_f_500.0kHz_w_25%,3.344176,3.60,132.834104,32.0,-199.831358,262.231724,32.0,...,-192.517826,103.330454,52.0,-153.974837,140.029578,51.0,-139.480175,127.249174,51.0,-119.617032
15,20221012,04_us_cont_20ms,150V_d_20.0ms_f_500.0kHz_w_0%,2.662949,1.00,37.242367,30.0,-142.503434,64.554166,32.0,...,-122.648603,35.783799,47.0,-106.552065,44.593004,47.0,-101.561787,45.370842,52.0,-89.226511
16,20221012,03_us_cont_test,sham,3.194684,2.12,1.172409,31.0,-10.139079,1.806094,35.0,...,-13.087529,2.752312,26.0,-13.702036,2.735144,27.0,-11.316440,2.481509,29.0,-10.847396
18,20221012,03_us_cont_test,150V_d_0.5ms_f_500.0kHz_w_25%,1.600897,2.68,5.214327,32.0,-29.725546,6.486290,34.0,...,-20.941946,3.116692,49.0,-21.966719,3.153741,51.0,-17.771443,3.508186,54.0,-18.428105
36,20221012,07_us_cont_500us,150V_d_0.5ms_f_500.0kHz_w_0%,3.280300,2.12,1.691382,89.0,-8.108532,2.161823,52.0,...,-4.021908,1.130528,86.0,-4.765848,1.648370,86.0,-4.316990,0.847763,85.0,-2.764988
38,20221012,07_us_cont_500us,150V_d_0.5ms_f_500.0kHz_w_25%,0.457030,3.60,0.739527,91.0,-3.825620,0.460416,12.0,...,0.260161,1.002697,93.0,0.587844,2.753027,84.0,-0.152257,3.423366,85.0,-0.878456
8,20221118,04_ex_click,40dB,1.905995,2.64,-1.398701,3.0,1.568414,-1.850112,66.0,...,0.070760,-1.293671,0.0,2.178231,-1.086629,3.0,2.272301,-0.459280,3.0,0.960247
11,20221118,03_us_cont_test2,sham,1.713978,2.88,-0.535055,48.0,0.329946,1.177518,74.0,...,-4.645785,3.462669,76.0,-6.949446,4.792778,88.0,-6.165477,3.729443,75.0,-5.390853
19,20221118,11_us_cont_20ms2,150V_d_20.0ms_f_500.0kHz_w_25%,2.343102,1.24,18.295739,52.0,-11.604909,33.094017,94.0,...,-21.936201,29.961704,48.0,-22.108287,21.667814,49.0,-21.949418,29.144438,48.0,-22.124389


In [62]:
group_deaf_data = target_deaf_data.groupby(by=["stim_params"])
group_intact_data = target_intact_data.groupby(by=["stim_params"])

In [66]:
deaf_mean_data = group_deaf_data.mean(numeric_only=True)
deaf_std_data = group_deaf_data.std(numeric_only=True)
deaf_size_data = group_deaf_data.size()
intact_mean_data = group_intact_data.mean(numeric_only=True)
intact_std_data = group_intact_data.std(numeric_only=True)
intact_size_data = group_intact_data.size()
deaf_mean_data
    

,date,abr_z_score,abr_latency,lfp_z_score_100,lfp_latency_100,lfp_ampliture_100,lfp_z_score_200,lfp_latency_200,lfp_ampliture_200,lfp_z_score_300,...,lfp_ampliture_500,lfp_z_score_600,lfp_latency_600,lfp_ampliture_600,lfp_z_score_700,lfp_latency_700,lfp_ampliture_700,lfp_z_score_800,lfp_latency_800,lfp_ampliture_800
stim_params,,,,,,,,,,,,,,,,,,,,,
150V_d_0.5ms_f_500.0kHz_w_0%,2.022112e+07,2.578360,2.786667,12.008401,49.666667,-8.361092,17.490523,37.333333,-19.257296,24.320827,...,-32.498292,14.271379,51.00,-27.943308,16.283958,50.000000,-22.184934,18.964359,51.666667,-19.196408
150V_d_0.5ms_f_500.0kHz_w_25%,2.022109e+07,1.059925,3.270000,16.608346,60.750000,-15.340681,17.812079,41.750000,-25.787926,13.009353,...,-28.346279,13.404281,66.00,-25.315647,14.094776,64.500000,-20.449910,16.176357,66.000000,-19.602566
150V_d_20.0ms_f_500.0kHz_w_0%,2.022112e+07,2.123538,2.100000,32.746234,41.750000,-74.127364,52.977497,49.750000,-97.859233,62.539830,...,-76.010396,29.295708,48.25,-62.689497,25.074808,45.250000,-52.751641,24.618253,45.750000,-46.659391
150V_d_20.0ms_f_500.0kHz_w_25%,2.022112e+07,2.769978,2.160000,64.546086,38.666667,-104.124001,116.754486,52.666667,-132.410899,135.471386,...,-103.640221,52.844474,47.00,-82.508468,60.910457,48.666667,-71.010680,58.231860,48.333333,-63.425031
40dB,2.022112e+07,1.634704,3.090000,1.140100,55.500000,-0.805145,1.371125,70.750000,-1.538772,3.227380,...,-0.664488,1.838757,43.50,-0.368538,2.479531,43.250000,0.100880,2.469646,51.500000,0.185080
sham,2.022114e+07,2.490755,2.232000,0.645029,32.400000,-2.516812,1.575226,49.000000,-4.044923,2.242408,...,-5.617393,2.442118,58.00,-6.346001,2.601072,61.200000,-4.911547,2.516100,49.200000,-4.670688


In [72]:
labels = ["(0.5, 0)", "(0.5, 50)", "(20, 0)", "(20, 50)", "Click 40dB", "sham"]

In [95]:
width = 0.3
os.makedirs("result/bar_plot_compare_deaf_intact",exist_ok=True)
for each_param in compare_parameters:
    deaf_error_data = deaf_std_data[each_param]/np.sqrt(deaf_size_data)
    intact_error_data = intact_std_data[each_param]/np.sqrt(intact_size_data)
    fig,ax=plt.subplots(dpi=600,figsize=(6,4))
    fig.set_facecolor("white")
    ax.bar(x=np.arange(-0.2,len(deaf_error_data)-1,1),height=deaf_mean_data[each_param],yerr=deaf_error_data,label="deaf",capsize=5,color="#ffffff",edgecolor="k",align="center",width=width)
    ax.bar(x=np.arange(0.2,len(intact_error_data),1),height=intact_mean_data[each_param],yerr=intact_error_data,label="intact",capsize=5,color="#bbbbbb",edgecolor = "k",align="center",width=width)
    ax.set_xticks(range(len(deaf_error_data)))
    ax.set_xticklabels(labels)
    ax.set_ylim(0,)
    ax.set_title(each_param)
    fig.subplots_adjust(right=0.8,left=0.1,top=0.9)
    plt.legend(bbox_to_anchor=(1.05, 1.0),loc="upper left")
    plt.savefig(f"result/bar_plot_compare_deaf_intact/{each_param}")
    plt.close()

C:\Users\smasa\AppData\Local\Temp/ipykernel_9836/2360891550.py:12: UserWarning: Attempting to set identical bottom == top == 0 results in singular transformations; automatically expanding.
  ax.set_ylim(0,)
